In [15]:
from bs4 import BeautifulSoup
import requests
import json
import os
import multiprocessing as mp
import random
import time # for time sleep
from tqdm import tqdm

In [2]:
base_url="https://dblp.org/db/conf/acl/index.html"
#Conference List Each Year
links_response = requests.get(base_url)
links=[]
if links_response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(links_response.text, 'html.parser')
    for link in soup.select('ul > li > cite > a.toc-link'):
        links.append(link.get('href'))
    soup.decompose()
    
#Crawling Papers in Each Conference
paper_links=[]
for link in links:
    link_response = requests.get(link)
    if link_response.status_code == 200:
        soup = BeautifulSoup(link_response.text, 'html.parser')
        temp_list=soup.select('ul > li.inproceedings > nav > ul > li > div > ul > li.ee > a')
        temp_list_len=len(temp_list)
        for i in range(0,temp_list_len):
            if temp_list[i].get('href').startswith('https://doi.org/'):
                paper_links.append(temp_list[i].get('href'))
        soup.decompose()
    elif link_response.status_code == 404:
        pass
    else:
        print("THERE IS AN ERROR!")
        break

In [ ]:
!pip install semanticscholar
from semanticscholar import SemanticScholar

In [ ]:
# paper doi만 추출
paper_DOI=[]
for paper_link in paper_links:
    paper_DOI.append(paper_link.split('https://doi.org/')[1])

## 약 10000개 paper 500개씩 Chunking & API CALL

In [74]:
# slice the list of paper doi
listOfPaper=[]
before=0
for i in range(500,len(paper_links),500):
    listOfPaper.append(paper_DOI[before:i])
    before=i
listOfPaper.append(paper_DOI[10000:len(paper_links)])

## 안에 있는 500개 중에 ERROR가 있어서 되는 chunk 먼저 API CALL

In [ ]:
sch = SemanticScholar()

## There is an error with API, so it needs to be modified a little bit
## 7~20 index first
temp7to9=[]
for i in range(7,10):
    paper=sch.get_papers(listOfPaper[i])
    temp7to9+=list(paper)
    
## 10,11,12,13,14 is ERROR...............................ㅠㅠㅠㅠ
temp15to17=[]  ## typo!! 16까지임
for i in range(15,17):
    paper=sch.get_papers(listOfPaper[i])
    temp15to17+=list(paper)
    
#17,18,19 ERROR
temp20=[]
paper=sch.get_papers(listOfPaper[20])
temp20+=list(paper)

In [468]:
SEARCH_FIELDS = [
        'abstract',
        'authors',
        'citationCount',
        'corpusId',
        'externalIds',
        'fieldsOfStudy',
        'influentialCitationCount',
        'isOpenAccess',
        'journal',
        'openAccessPdf',
        'paperId',
        'publicationDate',
        'publicationTypes',
        'publicationVenue',
        'referenceCount',
        's2FieldsOfStudy',
        'title',
        'url',
        'venue',
        'year'
    ]

In [ ]:
list11to13=[] #11~13번 chunk
for i in [11,12,13]:
    ls=[]
    for j in tqdm(range(0,500),desc=str(i)):
        doi=listOfPaper[i][j].split("10.18653/v1/")[1].upper()
        ls.append("10.18653/v1/"+doi)
    list11to13.append(ls)    

temp11to13=[]
for i in range(0,3):
    for j in tqdm(range(0,500),desc=str(i)):
        if j %10==0:
            time.sleep(20)
        try:
            paper=sch.get_paper(list11to13[i][j],SEARCH_FIELDS)
            temp11to13.append(paper)
        except:
            print(i,j,list11to13[i][j])

## 10.18653 이 아니라 10.3115인 구간으로 인해 (14,473)부터 다시 request

In [ ]:
list14=[] 
for j in tqdm(range(0,473),desc=str(14)):
        doi=listOfPaper[14][j].split("10.18653/v1/")[1].upper()
        list14.append("10.18653/v1/"+doi)
for j in tqdm(range(473,500),desc=str(14)):
    doi=listOfPaper[14][j].split("10.3115/v1/")[1].upper()
    list14.append("10.3115/v1/"+doi)

temp14=[]
for j in tqdm(range(0,500),desc=str(i)):
        if j %10==0:
            time.sleep(20)
        try:
            paper=sch.get_paper(list14[j],SEARCH_FIELDS)
            temp14.append(paper)
        except:
            print(j,list14[j])

In [ ]:
list17to19=[]
for i in [17,18]:
    ls=[]
    for j in tqdm(range(0,500),desc=str(i)):
        doi=listOfPaper[i][j].split("10.3115/")[1].upper()
        ls.append("10.3115/"+doi)
    list17to19.append(ls) 
    
ls=[]
for j in tqdm(range(0,260),desc=str(19)):
    doi=listOfPaper[i][j].split("10.3115/")[1].upper()
    ls.append("10.3115/"+doi)
k=23
for i in range(260,282):
    ls.append("10.1007/3-540-55801-2_"+str(k))
    k+=1
for j in tqdm(range(282,500),desc=str(19)):
    doi=listOfPaper[19][j].split("10.3115/")[1].upper()
    ls.append("10.3115/"+doi)
list17to19.append(ls)


temp17to19=[]
for i in range(0,3):
    for j in tqdm(range(0,500),desc=str(i)):
        if j %10==0:
            time.sleep(20)
        try:
            paper=sch.get_paper(list17to19[i][j],SEARCH_FIELDS)
            temp17to19.append(paper)
        except:
            print(i,j,list17to19[i][j])

In [ ]:
for j in tqdm(range(385,500),desc=str(10)):
        doi=listOfPaper[10][j].split("10.18653/v1/")[1].upper() 
        listOfPaper[10][j]="10.18653/v1/"+doi

temp10=[]

for j in tqdm(range(0,500),desc=str(i)):
        if j %10==0:
            time.sleep(20)
        try:
            paper=sch.get_paper(listOfPaper[10][j],SEARCH_FIELDS)
            temp10.append(paper)
        except:
            print(i,j,listOfPaper[10][j])

In [ ]:
url="https://api.crossref.org/works/"+listOfPaper[0][0]
paper_response=requests.get(url)
paper=paper_response.json()

### DOI 중에 arkiv로 되어있는 DOI로 찾아야하는 경우가 많음. 따라서 semantic scholar에서 논문 제목을 통해 paperId 추출

In [ ]:
paperIdList=[]
for i in [0,1,2,3,4,5,6]:
    ls=[]
    for j in tqdm(range(0,500),desc=str(i)):
            if j%10==0 and not (i==0 and j==0):
                time.sleep(20)
            url="https://api.crossref.org/works/"+listOfPaper[i][j]
            paper_response=requests.get(url)
            if paper_response.status_code==200:
                    paper=paper_response.json()
                    title=paper['message']['title'][0]
                    try:
                        temp=sch.search_paper(title)
                        if len(temp[0].paperId)==0:
                            ls.append(listOfPaper[i][j])
                            print(title,listOfPaper[i][j],i,j)
                        else:
                            ls.append(temp[0].paperId)
                            
                    except:
                        ls.append(listOfPaper[i][j])
                        print(title,listOfPaper[i][j],i,j)
            else:
                print("no")
    paperIdList.append(ls)
                        

In [ ]:
temp0to6=[]
for i in range(0,7):
    for j in tqdm(range(0,500),desc=str(i)):
        if j %10==0:
            time.sleep(20)
        try:
            paper=sch.get_paper(paperIdList[i][j],SEARCH_FIELDS)
            temp0to6.append(paper)
        except:
            print(i,j,paperIdList[i][j])

In [1120]:
acl_papers=[]

for temp in [temp0to6,temp7to9,temp10,temp11to13,temp14,temp15to17,temp17to19,temp20]:
    for i in range(0,len(temp)):
        temp[i]=dict(temp[i])
    acl_papers+=temp

datapath="./ACL_PAPERS.json"
with open(datapath,'w') as f:
    json.dump(acl_papers,f)

----

## 300개 missing data 중 55개 제외 살림 (abstract data 없는게 있음)

In [ ]:
c=0
absls=[]
for i in range(0,len(acl_papers)):
        if acl_papers[i]['abstract']==None:
            print(i, 'none')
            absls.append(i)
            c+=1
        elif len(acl_papers[i]['abstract'])<2:
            print(i,'dot')
            absls.append(i)
            c+=1
print(c)

In [ ]:
for i in absls:
    try:
        doi=acl_papers[i]['externalIds']['DOI'].split("10.18653/v1/")[1]
        link="https://aclanthology.org/"+doi
        link_response = requests.get(link)
    except:
        print(i,"no site")
        break
    if link_response.status_code == 200:
        soup = BeautifulSoup(link_response.text, 'html.parser')
        abstract=soup.select('#main >div>div> div >div> span')
        try:
            if len(abstract[0].text)!=0:
                acl_papers[i]['abstract']=abstract[0].text
        except:
            print(i,"no abstract")
            break
        soup.decompose()
    elif link_response.status_code == 404:
        print(i)

# 약 79~99 - ACM library

In [ ]:
for i in absls:
    try:
        doi=acl_papers[i]['externalIds']['DOI']
        link="https://dl.acm.org/doi/abs/"+doi
        link_response = requests.get(link)
    except:
        print(i,"no site")
        break
    if link_response.status_code == 200:
        soup = BeautifulSoup(link_response.text, 'html.parser')
        abstract=soup.select('div.abstractSection > p')
        try:
            if len(abstract[0].text)!=0:
                acl_papers[i]['abstract']=abstract[0].text
        except:
            print(i,"no abstract")
            break
        soup.decompose()
    elif link_response.status_code == 404:
        print(i)
    else:
        print("!!",i)

## 없는 데이터가 있다 .당연히 DOI가 모든 paper에 있을 줄 알고 doi.org/~ 된 것만 추가함...

In [ ]:
#Crawling Papers in Each Conference
missed_links=[]
for link in tqdm(links):
    link_response = requests.get(link)
    if link_response.status_code == 200:
        soup = BeautifulSoup(link_response.text, 'html.parser')
        temp_list=soup.select('ul > li.inproceedings > nav > ul > li > div > ul > li.ee > a')
        temp_list_len=len(temp_list)
        alreadyIn=0
        for i in range(0,temp_list_len):
            if temp_list[i].get('href').startswith('https://doi.org/'):
                alreadyIn=1
                break
        if alreadyIn==0:
            for i in range(0,temp_list_len):
                if temp_list[i].get('href').startswith('https://aclanthology.org/'):
                    missed_links.append(temp_list[i].get('href'))
        soup.decompose()
    else:
        print(link)
        

In [ ]:
for link in tqdm(links[136:]):
    link_response = requests.get(link)
    if link_response.status_code == 200:
        soup = BeautifulSoup(link_response.text, 'html.parser')
        temp_list=soup.select('ul > li.inproceedings > nav > ul > li > div > ul > li.ee > a')
        temp_list_len=len(temp_list)
        alreadyIn=0
        for i in range(0,temp_list_len):
            if temp_list[i].get('href').startswith('https://doi.org/'):
                alreadyIn=1
                break
        if alreadyIn==0:
            for i in range(0,temp_list_len):
                if temp_list[i].get('href').startswith('https://aclanthology.org/'):
                    missed_links.append(temp_list[i].get('href'))
        soup.decompose()
    else:
        print(link)

link_response = requests.get("https://aclanthology.org/W97-1005/")
if link_response.status_code == 200:
        soup = BeautifulSoup(link_response.text, 'html.parser')
        title=soup.select('#title > a')[0].text
        temp=soup.select('p.lead > a')
        authors=[{'name':i.text} for i in temp]
        year=soup.select('div.order-2 > dl > dd')[3].text

#for abstraction
def paperId(title):
    temp=sch.search_paper(title)
    if temp[0]['title']==title:
        return temp[0]
    else :
        print(title)
        return None

In [ ]:
missed_papers=[]
for link in tqdm(missed_links[2438:]):
    link_response = requests.get(link)
    if link_response.status_code == 200:
        soup = BeautifulSoup(link_response.text, 'html.parser')
        title=soup.select('#title > a')[0].text
        paper_info=paperId(title)
        if paper_info is None:
            continue
        else:
            missed_papers.append(paper_info)
            x=random.randint(10, 15)
            time.sleep(x)
        soup.decompose()
    else:
        print(1,link)

c=0
mls=[]
for i in range(0,len(missed_papers)):
        if missed_papers[i]['abstract']==None:
            print(i, 'none')
            mls.append(i)
            c+=1
        elif len(missed_papers[i]['abstract'])<2:
            print(i,'dot')
            mls.append(i)
            c+=1
print(c)
acl_papers+=missed_papers